<a href="https://colab.research.google.com/github/t1nh233/Pytorch-AI-Research-Roadmap/blob/main/Day_02_Autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- STARTUP BLOCK ---
import os
import torch
import numpy as np

# 1. Kết nối Google Drive (Để lấy dữ liệu)
from google.colab import drive
drive.mount('/content/drive')

# 2. Tạo đường dẫn tắt (Để code cho gọn)
# Lưu ý: Sửa lại đúng tên thư mục bạn tạo ở Bước 2 phần trên
DATA_PATH = '/content/drive/MyDrive/Pytorch_AI_Research_Bootcamp/datasets'
MODEL_PATH = '/content/drive/MyDrive/Pytorch_AI_Research_Bootcamp/checkpoints'

# 3. Kiểm tra GPU
if torch.cuda.is_available():
    print(f"✅ Đã bật GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ Cảnh báo: Bạn đang chạy bằng CPU. Hãy vào Runtime -> Change runtime type -> T4 GPU")

# 4. Tạo thư mục nếu chưa có (Tránh lỗi)
os.makedirs(DATA_PATH, exist_ok=True)
os.makedirs(MODEL_PATH, exist_ok=True)

print("🚀 Môi trường đã sẵn sàng!")

Mounted at /content/drive
✅ Đã bật GPU: Tesla T4
🚀 Môi trường đã sẵn sàng!


In [2]:
x = torch.tensor([2.0, 3.0], requires_grad=True)
y = x**2 + 3*x + 1
z = y.sum()
print(y)
print(z)

tensor([11., 19.], grad_fn=<AddBackward0>)
tensor(30., grad_fn=<SumBackward0>)


In [3]:
#Goi backward

z.backward(retain_graph=True)
print(x.grad)
z.backward()
print(x.grad)

tensor([7., 9.])
tensor([14., 18.])


In [4]:
#Vay neu toi boc tat ca trong no_grad() thi sao

with torch.no_grad():
  k = x**2 + 3*x + 1

print(y.grad_fn)
print(k.requires_grad)
print(k.grad_fn)
print(y.grad_fn.next_functions)

False
None
((<AddBackward0 object at 0x7d58fe631210>, 0), (None, 0))


In [23]:

##Chuan bi du lieu

raw_diamond = torch.tensor([[1.5, 0.8], [0.5, 0.4], [2, 0.9]], requires_grad=False)
print(raw_diamond)

diamond_tabular = raw_diamond / torch.max(raw_diamond)
print(diamond_tabular)

diamond_image = torch.randn((3, 1, 4, 4), requires_grad=False)
print(diamond_image)

diamond_priceR = torch.tensor([1500.0, 500.0, 2000.0])
print(diamond_priceR)

diamond_priceRnorm = diamond_priceR / diamond_priceR.max()
print(diamond_priceRnorm)

print("Data prepared!!")

tensor([[1.5000, 0.8000],
        [0.5000, 0.4000],
        [2.0000, 0.9000]])
tensor([[0.7500, 0.4000],
        [0.2500, 0.2000],
        [1.0000, 0.4500]])
tensor([[[[ 0.0798, -0.6742,  1.0694, -0.6828],
          [-1.5968, -0.6640, -0.9337,  0.1090],
          [-0.3594,  0.7387,  1.5235, -2.0768],
          [ 0.7439,  0.5472, -0.4189,  1.5414]]],


        [[[ 2.9314, -0.3435, -1.2611,  2.0426],
          [-0.4576, -0.9704, -0.0337, -0.6264],
          [-1.9236,  0.7808, -1.4058,  0.3680],
          [ 0.3715, -1.8181, -1.1867, -0.7527]]],


        [[[ 0.6548, -0.3124, -1.6507, -0.4286],
          [-1.3830,  0.4338, -0.7568,  1.7130],
          [-0.9779,  0.7017,  1.0497, -1.5550],
          [-1.4644, -0.3528,  0.7714,  0.3248]]]])
tensor([1500.,  500., 2000.])
tensor([0.7500, 0.2500, 1.0000])
Data prepared!!


In [24]:
##Viet ham autograd thu cong

class MagicSquare(torch.autograd.Function):
  @staticmethod
  def forward(ctx, input):
    ctx.save_for_backward(input)
    return input**2

  @staticmethod
  def backward(ctx, grad_output):
    input, = ctx.saved_tensors
    grad_input = 2 * input * grad_output
    return grad_input

def magic_square(input):
  return MagicSquare.apply(input)

In [25]:
##Build model va kiem tra graph

w1 = torch.tensor([100.0, 100.0], requires_grad=True)
w2 = torch.tensor([50.0], requires_grad=True)

def forward_pass(input_tabular, input_image):
  diaimage_mean = input_image.mean(dim=(1, 2, 3))
  val_image = magic_square(diaimage_mean * w2)

  val_tabular = torch.matmul(input_tabular, w1)
  return val_tabular + val_image

diamond_pred = forward_pass(diamond_tabular, diamond_image)
print(f"Giá dự đoán: {diamond_pred}")
print(f"Grad Fn: {diamond_pred.grad_fn}")

Giá dự đoán: tensor([125.8422, 224.3342, 247.0300], grad_fn=<AddBackward0>)
Grad Fn: <AddBackward0 object at 0x7d580750a410>


In [27]:
##Training loop

learning_rate = 0.01

print("Start training!!")
for epoch in range(100):
  diamond_predict = forward_pass(diamond_tabular, diamond_image)
  loss = (diamond_predict - diamond_priceRnorm).pow(2).mean()
  loss.backward()

  with torch.no_grad():
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad

    w1.grad.zero_()
    w2.grad.zero_()

  if epoch % 10 == 0:
    print(f"Epoch: {epoch}, Loss: {loss}")

print(f"Final Loss: {loss.item():.4f}")
print(f"w1 learned: {w1}")
print(f"w2 learned: {w2}")

Start training!!
Epoch: 0, Loss: 1292515.125
Epoch: 10, Loss: 640648.25
Epoch: 20, Loss: 488453.65625
Epoch: 30, Loss: 372415.0
Epoch: 40, Loss: 283942.84375
Epoch: 50, Loss: 216488.515625
Epoch: 60, Loss: 165058.921875
Epoch: 70, Loss: 125847.1015625
Epoch: 80, Loss: 95950.59375
Epoch: 90, Loss: 73156.3984375
Final Loss: 57310.8555
w1 learned: tensor([250.8177, 144.0256], requires_grad=True)
w2 learned: tensor([-1.4013e-45], requires_grad=True)
